Comparing the compute time for running the interior-point method on three different formulations of the optimization problem: (1) dual, (2) primal, and (3) modified primal.

In [8]:
using RCall
using Mosek
using JuMP
using CSV

In [2]:
L = Array{Float64,2}(CSV.read("../data/sample5000x20.txt", nullable = false, header = false, delim = ' '));

In [3]:
# vanilla interior point method for the primal
function vanilla_primal(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n);
    @constraint(mod, sum(x) == 1)
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x));
end

vanilla_primal (generic function with 1 method)

In [4]:
# vanilla interior point method for the primal
function vanilla_boxed(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, x[1:m] >= 0, start = 1/m);
    @NLobjective(mod, Min, -sum(log(sum(L[i,j]*x[j] for j = 1:m)) for i = 1:n)/n + sum(x[i] for i = 1:m));
    solve(mod);
    x = getvalue(x);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x));
end

vanilla_boxed (generic function with 1 method)

In [5]:
# vanilla interior point method for the primal
function vanilla_dual(L)
    n,m = size(L);
    mod = Model(solver=MosekSolver(QUIET = true));
    @variable(mod, y[1:n] >= 0, start = 1/n);
    @NLobjective(mod, Min, -sum(log(y[i]) for i = 1:n)/n);
    @constraint(mod, ic, L'*y .<= 1)
    solve(mod);
    x = -getdual(ic);
    x[x.< 1e-3] = 0;
    return sparse(x/sum(x));
end

vanilla_dual (generic function with 1 method)

In [6]:
function REBayes(L)
    @rput L;
    R"require(REBayes);
    t_rebayes = system.time(res <- KWDual(L, rep(1,dim(L)[2]), rep(1,dim(L)[1])/dim(L)[1]))[3];
    res$f[res$f < 1e-4] = 0
    x_rebayes = res$f / sum(res$f)"
    @rget x_rebayes;
    return sparse(x_rebayes);
end

REBayes (generic function with 1 method)

In [7]:
@time vanilla_dual(L)
@time vanilla_primal(L)
@time vanilla_boxed(L)
@time REBayes(L)

  7.692752 seconds (6.33 M allocations: 353.003 MiB, 1.71% gc time)
  1.469517 seconds (1.45 M allocations: 151.341 MiB, 15.19% gc time)
  0.808663 seconds (1.07 M allocations: 135.263 MiB, 24.32% gc time)


Loading required package: Matrix


  1.172427 seconds (149.93 k allocations: 8.076 MiB)


20-element SparseVector{Float64,Int64} with 3 stored entries:
  [1 ]  =  0.469905
  [13]  =  0.305865
  [14]  =  0.22423